# Exercise sheet \#4
## Using APIs
### Exercise 1
Write a Python script which gets the list of astronauts who are currently in Space. To do so, you can use the [astro.json](http://api.open-notify.org/astros.json) OpenNotify API. 

In [1]:
import requests
url = "http://api.open-notify.org/astros.json"
r = requests.get(url)
df = r.json()
l = list(map(lambda x : x['name'], df['people']))
l

['Sergey Prokopyev',
 'Dmitry Petelin',
 'Frank Rubio',
 'Nicole Mann',
 'Josh Cassada',
 'Koichi Wakata',
 'Anna Kikina',
 'Fei Junlong',
 'Deng Qingming',
 'Zhang Lu']

### Exercise 2

#### Question 2.1
Write a Python program, which, for each astronaut found in Exercise 1, retrieves  their (English) wikipedia article and extract the article's summary and links.

In [ ]:
!pip install wikipedia

In [ ]:
import wikipedia

summaries = []
links     = []
for astronaut in l:
    page = wikipedia.page(wikipedia.search(astronaut + " astronaut")[0])
    summaries.append(page.summary)
    links.extend(page.links)
print(summaries)
print(links)

#### Question 2.2
Extend your Python program so that it only keeps links that are pointing to wikipedia pages (in any language).

#### Question 2.3
Extend your Python program so that it processes these links as follows:
 - it retrieves the corresponding article and then extracts its references


#### Question 2.4
Extend your Python program to compute the average number of views for each astronaut's main article.

#### Question 2.5
Export the extracted information in a CSV file having the following fields:

`Astronaut's name ; Article's summary ; links separated by commas ; number of views`

### Exercise 3
#### Question 3.1 
Using Wikipedia, compile the list of UEFA's Intertoto cup winners sorted by country.

#### Question 3.2
Compute the number of Intertoto winners per country.

### Exercise 4

Let's play with New York Times API! Register here to get your API access token: https://developer.nytimes.com/apis

#### Question 4.1

Extract for each astronaut articles in NYT mentioning them from the year 2022. Create a dictionay where by the astronauts name you could get information on number of articles published about them in 2022, links to the last 10 articles of 2022, and list of keywords which are associated with those articles, including how many times each keyword appeared. 

For example 

    {"John Smith": {"articles": ["nytimes.com/The_Best_Astronaut",
                                 "nytimes.com/Do_we_really_believe_in_space_travel"],
                   "num_articles": 2,
                   "keywords": {"space": 2, "celebrities": 1}}}
                   
<span style="color:red">ATTENTION!</span> Never publish your API access key on github or any other open website! It is only to be used by you. (Well, otherwise the API service could ban you :) )

#### Question 4.2. Bonus :) 

What happens when we want to extract links to all the articles and not just the last 10? Extend your program to iterate through pages of the query results :)